In [11]:
import pandas as pd
import glob
import os
import pyodbc

C:\Users\Administrator\Anaconda3\envs\dash\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  after removing the cwd from sys.path.


In [20]:
def connect():
    usr = 'root'
    username = usr  # input("Enter username")
    password = ''  # getpass.getpass(prompt="Enter password")
    config = dict(
        server='132.203.62.111',  # change this to your SQL Server hostname or IP address
        port=1433,  # change this to your SQL Server port number [1433 is the default]
        database= 'dateaubase2020',
        username=username,
        password=password)
    conn_str = (
        'SERVER={server},{port};'
        + 'DATABASE={database};'
        + 'UID={username};'
        + 'PWD={password}')
    conn = pyodbc.connect(
        r'DRIVER={ODBC Driver 11 for SQL Server};'
        + conn_str.format(**config))
    cursor = conn.cursor()
    return cursor, conn
cursor, conn = connect()

InterfaceError: ('28000', "[28000] [Microsoft][ODBC Driver 11 for SQL Server][SQL Server]Login failed for user 'root'. (18456) (SQLDriverConnect); [28000] [Microsoft][ODBC Driver 11 for SQL Server][SQL Server]Login failed for user 'root'. (18456)")

In [4]:
path = "//10.10.10.13/infpc1_2/"

# List the .par files
listFile = glob.glob(path + '*.par')

# Sort the list from the oldest to the last
listFile.sort

# Index of the oldest file with new data (will change during the execution but the last file always has new data)
index = len(listFile) - 1
print(index, listFile)

3 ['//10.10.10.13/infpc1_2/2020-01-31_21-44-00.par', '//10.10.10.13/infpc1_2/2020-02-01_16-00-00.par', '//10.10.10.13/infpc1_2/2020-02-02_10-16-00.par', '//10.10.10.13/infpc1_2/2020-02-03_04-33-00.par']
